In [1]:
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np

### Select video

In [ ]:
VIDEO_PATH = os.path.join("..", "..", "..", "data", "processed", "real_life_deception_detection", "trial_lie_001.npy")
SELECTED_VIDEO = np.load(VIDEO_PATH, allow_pickle=True)

In [3]:
SELECTED_LANDMARKS = np.load(os.path.join('..', '..', '..', 'data', 'landmarks', 'combined_selected_points_emotions.npy'))

In [4]:
SELECTED_VIDEO = SELECTED_VIDEO[:, SELECTED_LANDMARKS, :]

## Display

In [5]:
# Assuming SELECTED_VIDEO has shape (1100, 478, 2)
num_frames, num_landmarks, _ = SELECTED_VIDEO.shape

# Create a color scale - one color per landmark
color_scale = px.colors.qualitative.Alphabet  # Using a larger color palette
landmark_colors = {i: color_scale[i % len(color_scale)] for i in range(num_landmarks)}

# Initialize figure with 3D and 2D subplots
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter'}]],
    subplot_titles=("3D Landmark Animation", "2D Landmark Positions")
)

# Add 3D scatter traces for each landmark
for landmark_idx in range(num_landmarks):
    # Get all coordinates for this landmark across frames
    x_coords = SELECTED_VIDEO[:, landmark_idx, 0]  # X coordinates
    y_coords = 1 - SELECTED_VIDEO[:, landmark_idx, 1]  # Flip Y axis for visualization
    frame_numbers = np.arange(num_frames) + 1  # Time axis
    
    fig.add_trace(
        go.Scatter3d(
            x=x_coords,
            y=frame_numbers,
            z=y_coords,
            mode='lines',
            name=f'Landmark {landmark_idx}',
            line=dict(color=landmark_colors[landmark_idx], width=2)
        ),
        row=1, col=1
    )

# Prepare frames for animation
frames = []
for t in range(num_frames):
    # Create a moving time plane
    plane_z = np.array([[0, 1], [0, 1]])
    plane_x = np.array([[0, 0], [1, 1]])
    plane_y = np.full_like(plane_x, t+1)  # Plane position matches current frame
    
    plane = go.Surface(
        z=plane_z,
        x=plane_x,
        y=plane_y,
        surfacecolor=np.full_like(plane_x, 0.5),
        colorscale=[[0, "grey"], [1, "grey"]],
        opacity=0.5,
        showscale=False,
        name="Time Plane"
    )
    
    # Get current frame's landmarks
    current_frame = SELECTED_VIDEO[t]
    x_coords = current_frame[:, 0]
    y_coords = 1 - current_frame[:, 1]  # Flip Y
    
    # Create scatter plot for current frame
    scatter = go.Scatter(
        x=x_coords,
        y=y_coords,
        mode='markers',
        marker=dict(
            size=6,
            color=[landmark_colors[i] for i in range(num_landmarks)],
            opacity=1
        ),
        name=f'Frame {t+1}'
    )
    
    frames.append(go.Frame(
        data=[plane, scatter],
        name=str(t+1)
    ))

# Add frames and animation controls
fig.frames = frames

# Update layout
fig.update_layout(
    height=800,
    scene=dict(
        xaxis_title='X coord',
        yaxis_title='Frame Number',
        zaxis_title='Y coord',
        xaxis=dict(range=[0, 1]),
        yaxis=dict(range=[1, num_frames]),
        zaxis=dict(range=[0, 1]),
    ),
    sliders=[{
        "steps": [{
            "args": [
                [str(t+1)],
                {
                    "frame": {"duration": 50, "redraw": True},
                    "mode": "immediate",
                    "transition": {"duration": 50}
                }
            ],
            "label": f"Frame {t+1}",
            "method": "animate"
        } for t in range(num_frames)],
        "currentvalue": {"prefix": "Frame: ", "visible": True, "xanchor": "center"},
    }],
    title="Landmark Positions Over Time",
    margin=dict(l=0, r=0, b=0, t=40),
    xaxis=dict(range=[0, 1], title="X coord"),
    yaxis=dict(range=[0, 1], title="Y coord")
)

# Add play/pause button
fig.update_layout(
    updatemenus=[{
        "type": "buttons",
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 50, "redraw": True},
                          "fromcurrent": True, "transition": {"duration": 50}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": True},
                          "mode": "immediate", "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "x": 0.1,
        "y": 0,
    }]
)

fig.show()